In [1]:
import torch
import torchaudiob

In [2]:
print(torch.__version__)
print(torchaudio.__version__)

2.2.0+cpu
2.2.0+cpu


In [18]:
import time 
import IPython
from typing import List

import matplotlib.pyplot as plt
from torchaudio.utils import download_asset
from torchaudio.models.decoder import ctc_decoder

In [5]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_10M
acoustic_model = bundle.get_model()

In [6]:
speech_file = download_asset('tutorial-assets/ctc-decoding/1688-142285-0007.wav')

In [8]:
waveform, sample_rate = torchaudio.load(speech_file)

In [10]:
if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, 
                                             bundle.sample_rate)

In [13]:
token = [label.lower() for label in bundle.get_labels()]
print(token)

['-', '|', 'e', 't', 'a', 'o', 'n', 'i', 'h', 's', 'r', 'd', 'l', 'u', 'm', 'w', 'c', 'f', 'g', 'y', 'p', 'b', 'v', 'k', "'", 'x', 'j', 'q', 'z']


In [14]:
from torchaudio.models.decoder import download_pretrained_files

files = download_pretrained_files('librispeech-4-gram')

100%|█████████████████████████████████████████████████████████████████████████████| 4.97M/4.97M [00:03<00:00, 1.55MB/s]
100%|███████████████████████████████████████████████████████████████████████████████| 57.0/57.0 [00:00<00:00, 23.5kB/s]
  1%|▋                                                                            | 25.1M/2.91G [00:09<12:28, 4.14MB/s]

limit_output extension: Maximum message size of 10000 exceeded with 10082 characters

KeyboardInterrupt: 

In [15]:
print(files)`

NameError: name 'files' is not defined

In [ ]:
LM_WEIGHT = 3.23
WORD_SCORE = -0.26

beam_search_decoder = ctc_decoder(
    lexicon_files = files,
    tokens = files.tokens,
    lm = files.lm,
    nbest = 3,
    beam_size = 1500,
    lm_weight = LM_WEIGHT,
    word_score = WORD_SCORE)

In [19]:
class GreedyCTCdecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        self.labels = lables
        self.blank = blank
        
    def forward(self, emission: torch.Tensor) -> List[str]:
        indices = torch.argmax(emission, dim=-1)
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        joined = "".join([self.labels[i] for i in indices])
        return joined.replaces("|", " ").strip().split()